## Normalizing the dataset along the row

A custom normalization transformer has been defined to perform the row normalization

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler


class RowScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaling_method='Standard'):
        self.scaling_options = ['Standard', 'MinMax', 'MaxAbs', 'Robust']
        assert (scaling_method in self.scaling_options), 'scaling_method:' + scaling_method + ' not in ' + str(self.scaling_options)
        self.scaling_method = scaling_method

        return None

    def fit(self, X, y=None):
        if self.scaling_method == 'Robust':
            self.scaler = RobustScaler()
        elif self.scaling_method == 'MinMax':
            self.scaler = MinMaxScaler()
        elif self.scaling_method == 'Standard':
            self.scaler = StandardScaler()
        elif self.scaling_method == 'MaxAbs':
            self.scaler = MaxAbsScaler()
        return self
    
    def transform(self, X, y=None):
        return self.scaler.fit_transform(X.transpose()).transpose()

In [ ]:
def printGridSearchResults(grid_search, scoring):

    print("Best parameters set found on development set:")
    print(grid_search.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = grid_search.cv_results_['mean_test_{}'.format(scoring)]
    stds = grid_search.cv_results_['std_test_{}'.format(scoring)]
    for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
        print("{:0.3f} (+/-{:0.03f}) for {}".format(mean, std * 2, params))

In [ ]:
import os

def save_model(model, model_dir, model_version):
    model_path = os.path.join(model_dir, model_version)
    if os.path.exists(model_path):
        print(model_path + ' already exist, not overwritten.')
    else:
        !mkdir -p {model_dir}
        joblib.dump(model, model_path)
        print(model_path + ' succesfully saved.')

def save_model(model, model_dir, model_version):
    model_path = os.path.join(model_dir, model_version)
    if os.path.exists(model_path):
        print(model_path + ' already exist, not overwritten.')
    else:
        !mkdir -p {model_dir}
        joblib.dump(model, model_path)

def save_keras_model(model, model_dir, model_version):
    model_path = os.path.join(model_dir, model_version)
    if os.path.exists(model_path):
        print(model_path + ' already exist, not overwritten.')
    else:
        !mkdir -p {model_dir}
        model.save(model_path)
        print(model_path + ' succesfully saved.')

In [ ]:
def rowScale(X):
    mean = tf.reshape(tf.reduce_mean(X, axis=1), [-1, 1])
    std =  tf.reshape(tf.math.reduce_std(X, axis=1), [-1, 1])
    return (X - mean)/std

In [ ]:
# class OneCycleScheduler(keras.callbacks.Callback):
#     def __init__(self, iterations, max_rate, start_rate=None,
#                  last_iterations=None, last_rate=None):
#         self.iterations = iterations
#         self.max_rate = max_rate
#         self.start_rate = start_rate or max_rate / 10
#         self.last_iterations = last_iterations or iterations // 10 + 1
#         self.half_iteration = (iterations - self.last_iterations) // 2
#         self.last_rate = last_rate or self.start_rate / 1000
#         self.iteration = 0
#     def _interpolate(self, iter1, iter2, rate1, rate2):
#         return ((rate2 - rate1) * (self.iteration - iter1)
#                 / (iter2 - iter1) + rate1)
#     def on_batch_begin(self, batch, logs):
#         if self.iteration < self.half_iteration:
#             rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
#         elif self.iteration < 2 * self.half_iteration:
#             rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
#                                      self.max_rate, self.start_rate)
#         else:
#             rate = self._interpolate(2 * self.half_iteration, self.iterations,
#                                      self.start_rate, self.last_rate)
#             rate = max(rate, self.last_rate)
#         self.iteration += 1
#         K.set_value(self.model.optimizer.lr, rate)